# FEATURE MIMIC ATTACK + DEFENSE


In this notebook there is a training of the defense baseline models and an analysis of its results. At the end of the notebook there are two sections where the code for the attack and the defense is presented. There is also the code to make the plots (heatmaps and histograms).

In [ ]:
import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping

In [ ]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

# Lista para cambiar los paths rapido.
workers = ["Victor","Alex", "Local"]

# Cambiar el numero aqui para cambiar los paths.
worker = workers[0]

if worker == "Victor":
  path= "/content/gdrive/MyDrive/Q7/CBI/ANN"
elif worker == "Alex":
  path = "/content/gdrive/MyDrive/CBI/ANN"
else: path = os.getcwd()

## Lectura de datos

In [ ]:
data_path=path+"/newTrainingData"
file_name="training_data.csv"
df_train=pd.read_csv(data_path+'/'+file_name)

file_name="testing_data.csv"
df_test=pd.read_csv(data_path+'/'+file_name)

print("DF train shape:",df_train.shape)
print("DF test shape:",df_test.shape)

global distances 
distances = [80*i for i in range(1,26)]

# Conversion de los datos

In [ ]:
global sc_input, sc_output
sc_input = MinMaxScaler()
sc_output = MinMaxScaler()


def modify_data(df, train):
  # Function that converts the provided data frame into target and explicative variables.
  colnames=df.columns[1:25]
  X=df.iloc[:,1:25]
  Y=df.iloc[:,0].to_numpy().reshape(-1,1)
  if train: 
    X = sc_input.fit_transform(X)
    Y = sc_output.fit_transform(Y)
  else: 
    X = sc_input.transform(X)
    Y = sc_output.transform(Y)
  return X,Y

In [ ]:
X_train,Y_train = modify_data(df_train, True)
# X_test, Y_test = modify_data(df_test, False)

# Creacion del modelo

In [ ]:
def ann():
    model = Sequential()
    model.add(Dense(12, input_dim = 24, activation = 'tanh'))
    model.add(Dense(6,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model

# Entrenamiento del modelo

In [ ]:
def generate_and_train(X_train,Y_train):
  model_ann = KerasRegressor(build_fn=ann,epochs=500,batch_size=32)

  start_time = time.time()
  callback = EarlyStopping(monitor='loss', patience=500)
  model_ann.fit(X_train, Y_train, callbacks=[callback])
  time_train_ann = time.time() - start_time
  return model_ann

In [ ]:
model_ann = generate_and_train(X_train,Y_train)

In [ ]:
pickle.dump(model_ann,open(path+"/trainedModel/new_model.pkl","wb"))

# Resultados de test

In [ ]:
def test_results(model,X_test,Y_test,log = True): 
  start_time = time.time()
  Y_test_pred=model.predict(X_test)
  time_eval_ann=time.time()-start_time

  real=list(list(zip(*Y_test))[0])

  #real=list(map(list, zip(*Y_train)))
  pred=list(Y_test_pred)
  print(pred)

  plt.plot(real,pred,'bo')
  x= np.linspace(0,1,100)
  plt.plot(x,x,"-r")
  plt.show()
  plt.close

  tot_abs, tot_square = 0, 0
  for i in range(len(real)):
    tot_abs += (real[i] - pred[i])
    tot_square += (real[i] - pred[i])**2
  if log:
    print('MAE:', tot_abs/len(real))
    print('MSE:', tot_square/len(real))
  return Y_test_pred

In [ ]:
path_modified = path+'/modifiedData/GD/alpha1.csv'
df_modified = pd.read_csv(path_modified)

In [ ]:
error_matrix_alpha_1_GD = []
for i in range(24):
  interest = 25-(i+1)
  idx1 = sum([x for x in range(interest+1,25)])*25
  idx2 = idx1 + interest*25
  points = df_modified.iloc[idx1:idx2,]
  X_mod, Y_mod = modify_data(points, False)
  aux = test_results(model_ann,X_mod,Y_mod,log = False)
  aux = sc_output.inverse_transform(aux.reshape(1,-1))[0]
  tar = sc_output.inverse_transform(Y_mod.reshape(1,-1))[0]
  mean_error = [0 for _ in range(interest)]
  for j in range(interest):
    elements = aux[j*25:j*25+25]
    targets = tar[j*25:j*25+25]
    err = 0
    for k in range(25):
      err += np.abs(elements[k]-targets[k])
    mean_error[j] = err/25
  error_matrix_alpha_1_GD.append(mean_error)

In [ ]:
pickle.dump(error_matrix_alpha_1_GD,open(path+"/modifiedData/GD/EM_alpha0.5.pkl","wb"))

In [ ]:
path_modified = path+'/modifiedData/Z/alpha1.csv'
df_modified = pd.read_csv(path_modified)

In [ ]:
error_matrix_alpha_1_GD = []
for i in range(24):
  interest = 25-(i+1)
  idx1 = sum([x for x in range(interest+1,25)])*25
  idx2 = idx1 + interest*25
  points = df_modified.iloc[idx1:idx2,]
  X_mod, Y_mod = modify_data(points, False)
  aux = test_results(model_ann,X_mod,Y_mod,log = False)
  aux = sc_output.inverse_transform(aux.reshape(1,-1))[0]
  tar = sc_output.inverse_transform(Y_mod.reshape(1,-1))[0]
  mean_error = [0 for _ in range(interest)]
  for j in range(interest):
    elements = aux[j*25:j*25+25]
    targets = tar[j*25:j*25+25]
    err = 0
    for k in range(25):
      err += np.abs(elements[k]-targets[k])
    mean_error[j] = err/25
  error_matrix_alpha_1_GD.append(mean_error)

In [ ]:
sc_output.inverse_transform(Y_test.reshape(1,-1))[0]

In [ ]:
print("RESULTS FOR ORIGINAL DISTANCE:", og_dist)
a = test_results(model_ann,X_mod,Y_mod)

# Analisis de los datos

In [ ]:
data_path=path+"/contodo"
file_name="training_data.csv"
df_train=pd.read_csv(data_path+'/'+file_name)

file_name="testing_data.csv"
df_test=pd.read_csv(data_path+'/'+file_name)

print("DF train shape:",df_train.shape)
print("DF test shape:",df_test.shape)

global distances 
distances = [80*i for i in range(1,26)]

In [ ]:
%%capture
!pip install shap
import shap

# print the JS visualization code to the notebook
shap.initjs()

In [ ]:
def var_boxplot(df):
  total_items = len(df.columns)
  items_per_row = 4
  total_rows = math.ceil(total_items / items_per_row)


  fig = make_subplots(rows=total_rows, cols=items_per_row)

  cur_row = 1
  cur_col = 1

  for index, column in enumerate(df.columns):
      fig.add_trace(go.Box(y=df[column], name=column), row=cur_row, col=cur_col)
      
      if cur_col % items_per_row == 0:
          cur_col = 1
          cur_row = cur_row + 1
      else:
          cur_col = cur_col + 1
      fig.update_layout(height=1000, width=750,  showlegend=False)
  fig.show() 

In [ ]:
def line_plots_and_params(df,pol_deg,plot = True):
  total_items = len(df.columns)
  items_per_row = 3
  total_rows = math.ceil(total_items / items_per_row)

  fig = make_subplots(rows=total_rows, cols=items_per_row, subplot_titles=df.columns)

  cur_row = 1
  cur_col = 1

  parameters = {}

  for index, column in enumerate(df.columns):
      fig.add_trace(go.Scattergl(y=df[column], 
                              x=df['dist'], 
                              mode="markers", 
                              marker=dict(size=3)), 
                    row=cur_row, 
                    col=cur_col)
      
      fit = np.polyfit(df['dist'], df[column],pol_deg)
      poly = np.poly1d(fit)
      intercept = poly(np.unique(df_train['dist']))
      parameters[column] = poly
      
      fig.add_trace(go.Scatter(x=np.unique(df["dist"]), 
                              y=intercept, 
                              line=dict(color='red', width=1)), 
                    row=cur_row, 
                    col=cur_col)
      
      if cur_col % items_per_row == 0:
          cur_col = 1
          cur_row = cur_row + 1
      else:
          cur_col = cur_col + 1
      fig.update_layout(height=2500, width=1850, showlegend=False)
  if plot: fig.show()
  return parameters

In [ ]:
params = line_plots_and_params(df_train,5,plot=True)

# Analisis del modelo 

In [ ]:
names = df_test.columns[1:]
explainer = shap.KernelExplainer(model_ann.model, X_train[:100])
shap_values = explainer.shap_values(X_test[:100])
shap.summary_plot(shap_values, X_test, plot_type='bar',feature_names=names )

Importancia de las features en la prediccion.

In [ ]:
def shap_plot(obs):
  p = shap.force_plot(explainer.expected_value, shap_values[obs][0], X_test[0],feature_names = names)
  return p
  
shap.initjs()
# Esta es la prediccion para la primera obs.
shap_plot(0)

El baseline value es la media de las predicciones. Las variables en rojo empujan la media hacia arriba y las azules la disminuyen.

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0], X_test[:100],feature_names = names)

# Ataque

In [ ]:
def find_params(alpha,pol_deg,df):
  '''
  Function that finds the polynomial fit
  parameters for the alpha*data points.

  Input:
        -alpha: Percentage of traning data available per distance.
        -pol_deg: Degree of the polynomial used to approximate the variables' distribution functions.
        -df: Pandas dataframe that contains the training data.
  Output:
        -Parameters.
  '''
  # 1. Selecting the data observations.
  aux_df = []
  for dist in distances:
    n_points = math.floor(len(df[df.dist == dist])*alpha)
    aux_df.append(df[df.dist == dist].sample(n = n_points))
  selected_obs = pd.concat(aux_df)
  # 2. Obtaining the parameters of the approximation.
  params = line_plots_and_params(selected_obs,pol_deg,False)
  return params

In [ ]:
def attack(obs,poly_params,target,model,method, res=False):
  '''
  Function that modifies the provided obs according to 
  the polynomial approximation to mimic the desired target.

    Input:
        -obs: Observation to modify.
        -poly_params: Didctionary containing a polynomial approximation for each feature.
        -target: Distance to mimic.
        -model: Model to compute the distance given a set of features.
  Output:
        -MAE between the target and the prediction obtained by the model.
  '''
  og_dist = obs.dist
  for i,(col,pol) in enumerate(poly_params.items()):
    if col != "dist":
      if (method == "All" or (method == "Cov" and col.split("_")[0] == "sigma") 
          or (mehtod == "Var" and len(col.split("_")[1]) == 2 and 
          col.split("_")[1][0] == col.split("_")[1][1])):
            v1 = pol(target)
            v2 = pol(og_dist)
            obs[i]+= (v1-v2)

  if res:
      return obs

  pred = model.predict(sc_input.transform(obs[1:].values.reshape(1,-1)))
  pred = sc_output.inverse_transform(pred.reshape(1,-1))[0][0]

  return np.abs(target-pred)

In [ ]:
n_dist = len(distances)
results = {}
results_list = {}
alphas = [1,0.25,0.1]
polynomial_degrees = [1,3,5]
methods = ["All", "Cov", "Var"]

for alpha in alphas:
    for pol in polynomial_degrees:
        for method in methods:
            # Find the optimal parameters for the combination of variables
            params = find_params(alpha, pol, df_train)

            # Attack for each distance
            error_matrix = np.zeros((n_dist, n_dist))
            for i,obs_dist in enumerate(distances):
                for j,target_dist in enumerate(distances):
                    obs_dist_i = df_test[df_test.dist == obs_dist]
                    for _, obs in obs_dist_i.iterrows():
                        error_matrix[i,j] += attack(obs, params, target_dist, model_ann, method)
                    error_matrix[i,j] /= len(obs_dist_i)
            # Imprimimos la matrix por si falla el pickle.
            name = str(alpha) + str(pol) + method
            results[name] = error_matrix
            results_list[name] = error_matrix.tolist()

In [ ]:
import pickle

try: 
    with open('filename.pickle', 'wb') as handle:
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
except:
    with open('filename.pickle', 'wb') as handle:
        pickle.dump(results_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Defensa

In [ ]:
mean = df_train.groupby(by='dist').mean().reset_index().reset_index().drop(columns='index')
std = df_train.groupby(by='dist').std().reset_index().reset_index().drop(columns='index')

In [ ]:
import random
import time

diff_dist_bad = {}
diff_dist_tot = {}

def normalize(row):
    mean_row = mean[mean['dist'] == row['dist']]
    std_row = std[std['dist'] == row['dist']]

    aux = abs((row - mean_row) / std_row)
    return aux.iloc[0]
    

def transform_data(row):
    i = row['dist']/80
    i = random.randint(i + 1, 25)
    i = 25
    
    res =  attack(row, params, i*80, model_ann, 'All', res=True)
    res['dist'] = i*80
    return res

params = find_params(1, 5, df_train)
iterations = 1
boundaries = [0.96]
f1 = []

aux_not_mod = []
aux_mod = []

for boundary in boundaries:

    v, w = 0, 0

    for i in range(iterations):
        df_test_mod = df_test[df_test['dist'] == 80]
        df_test_mod = df_test_mod.apply(transform_data, axis=1)

        test_norm = df_test[df_test['dist'] == 2000].apply(normalize, axis=1)
        test_norm = test_norm.drop('dist', axis=1)
        aux = test_norm.mean(axis=1)
        v += len(aux[aux >= boundary]) / len(test_norm)
        aux_not_mod += aux.to_list()

        test_norm_mod = df_test_mod.apply(normalize, axis=1)
        test_norm_mod2 = test_norm_mod.drop('dist', axis=1)
        aux = test_norm_mod2.mean(axis=1)
        w += len(aux[aux >= boundary]) / len(test_norm_mod2)
        aux_mod += aux.to_list()

        for idx in range(len(aux)):
            act_dist = (idx//25 + 1) * 80
            diff_dist = df_test_mod['dist'].iloc[idx] - act_dist

            if aux.iloc[idx] < boundary:
                if diff_dist in diff_dist_bad:
                    diff_dist_bad[diff_dist] += 1
                else:
                    diff_dist_bad[diff_dist] = 1

            if diff_dist in diff_dist_tot:
                diff_dist_tot[diff_dist] += 1
            else:
                diff_dist_tot[diff_dist] = 1



    tp = w/iterations
    fp = v/iterations
    f1sc = tp/(tp + 0.5*(fp + (1 - tp)))
    f1.append(f1sc)
    print(boundary, ':', tp, fp, f1sc)

In [ ]:
mean = df_train.groupby(by='dist').mean().reset_index().reset_index().drop(columns='index')
std = df_train.groupby(by='dist').std().reset_index().reset_index().drop(columns='index')

params = find_params(1, 4, df_train)

def normalize(row):
    mean_row = mean[mean['dist'] == row['dist']]
    std_row = std[std['dist'] == row['dist']]

    aux = abs((row - mean_row) / std_row)
    return aux.iloc[0]

def transform_data(row, j):    
    res =  attack(row, params, j*80, model_ann, 'All', res=True)
    res['dist'] = j*80
    return res


matrix = [[0 for _ in range(24 - j)] for j in range(24)]

for i in range(0, 24):
    res = []
    for j in range(0, 24 - i):
        df_test_mod = df_test[df_test['dist'] == (i+1)*80]
        df_test_mod = df_test_mod.apply(transform_data, axis=1, args=((i + j + 2),))
        df_test_mod = df_test_mod.apply(normalize, axis=1).mean(axis=1)
        matrix[i][j] += sum(df_test_mod < 0.96)/len(df_test_mod)

In [ ]:
# Code for heatmap

import plotly.graph_objects as go
global distances 
distances = [80*i for i in range(1,26)]
fig = go.Figure(go.Heatmap(z = "INSERT HERE MATRIX OF HEATMAP", x = distances, y = distances, zmax = 1, zmin = 0, zauto= False))
fig.update_layout(width=600)

In [ ]:
import numpy as np
import random
from matplotlib import pyplot as plt

# fixed bin size
bins = np.arange(0, 2500, 80) # fixed bin size

plt.xlim([min(diff_dist)-80, max(diff_dist)+160])

plt.hist(diff_dist, bins=bins, alpha=0.5)
plt.title('Difference in distances (actual vs mimicked)')
plt.xlabel('Difference')
plt.ylabel('Count')

plt.show()

In [ ]:
import numpy as np

def optimal_boundary(a, b, center, step):
    best_boundary, best_f1, best_tp, best_fp = 0, 0, 0, 0
    boundaries = list(np.arange(center - 10*step, center + 11*step, step))
    for boundary in boundaries:
        tp, fp, fn = 0, 0, 0
        for i in range(len(a)):
            if b[i] >= boundary:  # tp
                tp += 1
            elif b[i] < boundary: # fn
                fn += 1
            if a[i] >= boundary:  # fp
                fp += 1
        f1 = tp/(tp + 0.5*(fp + fn))
        if f1 > best_f1:
            best_f1 = f1
            best_boundary = boundary
            best_tp = tp
            best_fp = fp
    return best_boundary, best_f1, best_tp/len(b), best_fp/len(a)


def transform_to_vectors(data):
    no_modified = []
    modified = []

    for e in data:
        modified += e[0]
        no_modified += e[1]
    return no_modified, modified

In [ ]:
# EXAMPLE OF RESULT OF EXECUTION (JUST ONE ITERATION)

supermega = [[[1.550117690356374, 3.087847566991573, 2.8721309511296593, 2.898969443632914, 3.295381886227255, 3.0517454302385705, 3.1901499602204604, 3.0497510854241883, 3.690724364330953, 2.9345704312732783, 3.754358626480344, 3.7786134936715854, 3.2320213537773856, 3.1590615793796375, 3.8075998340934096, 2.8924618894234038, 2.8485882373060134, 3.4922723874731023, 2.3208611503051735, 3.44508793062596], [1.6868574304250386, 1.6844346589971073, 1.7079895350464838, 1.8157998439506624, 1.7012993619523666, 1.7364361016037266, 1.7322543358849711, 1.6622787415320337, 1.7102583444552937, 1.7777583225122395, 1.6574035445105206, 1.6813470062445623, 1.7102385075560615, 1.66479447863231, 1.6803200505869762, 1.6453232788194525, 1.744808521205272, 1.6523982835863362, 1.6745975481758546, 1.703833631031032]],
[[3.269224291859492, 3.8508863644109588, 2.9292375133771404, 3.604773751256618, 3.8606753908947162, 3.13980544970111, 2.727855273280543, 2.88998391771819, 1.9965886873755745, 3.569632156282335, 2.0470933039387194, 3.1417003338104417, 2.290072524521911, 2.9124846422794564, 2.040925133385794, 2.0220660174253835, 3.973050153489847, 2.5103705384347657, 2.9968408146903154, 3.3621222989319075], [1.7455493857898963, 1.6664042299819608, 1.7210208453277325, 1.69522500248903, 1.7255583258041813, 1.7562756193160456, 1.7076679470047857, 1.7553591860433158, 1.7078563501711461, 1.7113406299351235, 1.7072374970362183, 1.7059700919264393, 1.9990887284738743, 1.7069086814323524, 1.7035309557683223, 1.775724036481824, 1.7455914023065058, 1.730837593806599, 1.7612038376096941, 1.7913686408270224]],
[[3.6146977991019784, 2.7198909679972614, 2.5834774494625963, 2.723855408634936, 3.363595866711847, 2.2091864033701816, 2.83259515436596, 2.6654929945270176, 2.6580264010175374, 2.6834491568370216, 3.4784367290669542, 3.8438166206663316, 2.1924155565110937, 3.529953921854446, 3.3683013153685826, 2.891395743900066, 2.5644946348214512, 2.1629458193174784, 2.478610004940726, 2.131018083767323], [1.8287545407713188, 1.7417820835813367, 1.7852066566698332, 1.7291208507497788, 1.7896026170968404, 1.7823063401265342, 1.774434146938509, 1.7195827296300128, 1.7148286279725935, 1.7436481049039796, 1.8091772386903413, 1.7251182566493697, 1.7415786384445038, 1.735995603235244, 1.7208530792604568, 1.8846056283283783, 1.8217969917125154, 1.843285491464093, 1.8297688891427226, 1.8043460985109385]],
[[3.743111267087824, 2.819637432550919, 3.3452444803410133, 2.695400580749668, 2.491461780932545, 3.309911256196211, 3.067418506945265, 2.7458687069924808, 3.155818226690307, 3.2862412288821834, 2.6619093254767385, 3.508428797308889, 2.6503898586898345, 2.705485521530677, 2.5604648699029013, 2.451808420134161, 2.393862107776593, 3.1720219394121614, 2.3483545334750504, 2.6366094324156006], [1.7732689388780238, 1.8669535015379228, 1.8183785196083169, 1.8777714094225408, 1.8193172905994996, 1.8085010339614114, 1.785790996305497, 1.7982253427458037, 1.8030487107520878, 1.8526774923251512, 1.789872864209094, 1.8021457924335036, 1.7558195548184843, 1.816069194338681, 1.7962506703510763, 1.8358317414610066, 1.7731368514992367, 1.830378084926068, 1.767464382174192, 1.7471171755743387]],
[[2.3244066295325343, 3.4826960901148425, 3.0694473373742994, 3.275791225176574, 3.244831288822188, 2.330904572170939, 2.09244358681067, 2.6758872075389952, 2.8612825235380384, 3.3679716893909633, 2.785366080708332, 3.0010711726326234, 3.2305691572122073, 3.335913427070251, 3.2929333509957304, 2.3169944063517436, 3.3704137874923616, 3.1654244286339637, 2.297881179783513, 2.597326878368471], [1.8278803764055962, 1.8412568246923484, 1.768173247551197, 1.7928189463090582, 1.7605888109552694, 2.061319538176531, 1.7687321972004117, 1.7740916408504437, 1.8302302064419802, 1.828538207985183, 1.799883177129087, 1.782014066412227, 1.76959343095943, 1.8343217644267147, 1.8177878233646914, 1.7976239722327738, 1.8446039375505234, 1.815428958018893, 1.7994027487472495, 2.08024407783045]],
[[3.3426252541361756, 2.183817484161234, 3.343841992851817, 3.2100929854041462, 3.3624425976941166, 2.9989096145176, 3.110590110011593, 2.1776556113981473, 3.0378165373943022, 3.0110851547847988, 3.2594910376983965, 2.930882415346473, 2.650309827392649, 3.0426977720281543, 3.0472895846265518, 3.22393401745955, 3.2665950058260917, 3.17267549332258, 3.14703117373113, 2.774387891040997], [1.8026465733317645, 1.7973891327123772, 2.097198024036353, 1.8157624111366502, 1.9573143008022884, 1.8044503542378274, 1.8396870782570898, 1.8902382654755137, 1.8649392655896748, 1.8666900858657804, 1.8674639218491784, 1.82730258803524, 1.8242422993278136, 1.8061965004055338, 1.8349684373346795, 1.8036574467503095, 1.8411810405342286, 1.8102657155213238, 1.8199555507165466, 1.8707793665996493]],
[[3.2474656661897963, 2.813556625536514, 2.2268759387814416, 2.674047449313436, 3.2888106497688567, 2.7632463477110765, 2.954176200601237, 2.5868376087858476, 2.6188984344715327, 2.5989568626126434, 2.706711696664304, 2.6284111668778567, 2.2248235155977536, 3.2492190234130596, 2.1999589643800377, 2.7722313257075473, 3.2806193657858183, 2.8614848315523957, 2.9276683357562536, 2.5547036018209095], [1.8363482808394234, 1.8674900350415398, 1.8460327352544794, 1.869855384926531, 1.9813296089944363, 1.8646640556904586, 1.8863952130259862, 1.8851302417270623, 1.9149169672605229, 1.90628881892675, 1.9017935321677346, 1.8996490034928564, 1.8694142089967196, 1.9049792689214997, 1.8418012206319687, 1.8591840883957782, 1.83981880365066, 1.9018300124260137, 1.8880869896376966, 1.8527329620109936]],
[[2.8053378761004706, 2.2107210805671533, 2.1184453361226545, 3.092407391734762, 2.1939043102746894, 3.272143898344025, 2.864897387584853, 2.629348006203625, 2.150588363283383, 2.785039974519187, 2.5706506554503306, 2.8384207616107915, 2.2482807832695815, 2.247456089498327, 2.2335326147992145, 2.258509366738884, 2.1891508491296543, 2.1627171782556136, 2.9065041894140413, 2.5800870458360485], [1.9418805654227882, 1.9707718283669091, 1.8640997567618665, 1.8644469751188852, 1.8927008708920838, 1.8833047712385471, 1.860981801220962, 1.8562272131429736, 1.8615347167494019, 1.872025350457621, 1.933352136520706, 1.91013034013625, 1.907972162786138, 1.8839892136675562, 2.12210923473626, 1.9234032571310609, 1.8610102142938385, 2.093998820292935, 1.9062840966118093, 1.8661623868516093]],
[[2.5001389397229534, 2.9597267663934153, 3.073906252555749, 2.96536158977408, 3.203416336395816, 2.8881501051860234, 2.476900095357345, 3.0212719583368575, 2.2522863253506147, 2.2151140559806377, 2.2288002664931326, 3.475280954255033, 2.2143271594101397, 2.8181631775660696, 2.2544621264320255, 2.4592348783529685, 3.0562572204906364, 2.1959175545499385, 2.2880022879297295, 2.9157698386688367], [1.924264199839854, 1.9127683993722588, 1.8892181663929593, 1.9519299666204455, 1.8901017444614114, 1.8934156785844947, 1.9416881440060254, 1.8909163328227998, 1.9001840880232805, 1.9239965801875487, 1.8604563799221736, 1.9785146030737593, 1.860231958024408, 1.9177131802220038, 1.9554115932976788, 1.9242193686679272, 1.8898632376577478, 1.8471243455477255, 1.8726631123899857, 1.946113049749112]],
[[2.962709635774312, 2.499050408673498, 2.860685222399034, 2.897722437857436, 2.36267197705891, 1.765346881137854, 2.859869723620016, 2.7544590119982697, 2.953386253890956, 2.250846198215363, 2.873802038230249, 2.6338064940212225, 2.731325557913791, 2.359450732582878, 2.6782751274987406, 2.859000433629549, 2.8535971186243243, 2.3439840759044857, 2.685700583770964, 2.6476319994503434], [1.9256033583904164, 1.895953073734005, 1.9591497944753644, 1.9403422064158071, 1.9247388839964032, 1.9341062138706273, 1.856365859113344, 1.8975164433460052, 1.9306534642869104, 1.939615773084976, 1.9315880609229865, 2.002773346323914, 1.9486355267130073, 1.977357679794975, 1.9568171675639383, 1.9889140246030153, 1.9257341766321057, 1.922786138884886, 1.992976640048677, 1.9058228492070821]],
[[2.6205092451872813, 2.409397733971112, 2.695814211764617, 2.3513688074640644, 2.4751847832097837, 2.6289227364705687, 2.340063925704235, 2.3505435133102024, 3.123422375408842, 2.7554511275179454, 2.2156565021276315, 2.5050772190723345, 2.4511165272232103, 2.5051753689972958, 2.4061633471209367, 2.338532404645576, 2.428076002145407, 2.3902629304235488, 2.473641040040758, 2.5217037754869467], [1.949910871532078, 1.9375170779416622, 1.9861988846233438, 1.9337266143932887, 1.9757869249723072, 1.9207211834371518, 1.9565436932907907, 1.9248330083869107, 2.0870064180156294, 1.941673680072913, 2.009039036412696, 1.9681045458656645, 2.0801093529523786, 1.9695598509267351, 1.9828411333244857, 1.972749768080799, 1.934327199111512, 1.9648912704289798, 1.959402249637265, 1.97145142394128]],
[[2.32378548620667, 2.361559780502868, 2.5827882462212775, 2.2560140302711207, 2.603731879241214, 2.4626170803736986, 2.4584025260761257, 2.8294305848322345, 2.8738488150646666, 2.4133894377061096, 2.7707385734725922, 2.6136627187791945, 2.4590004705723145, 2.3543095327728256, 2.6902543234835052, 1.8173750791513832, 2.4344844426945293, 2.434573548032734, 2.3545619981005994, 2.4360732703037105], [1.9740046043936068, 2.0746942131749533, 1.9157267775564486, 1.9466753427856418, 1.9800584071764031, 1.9361372115268587, 2.00607667945818, 1.944991764281288, 2.01974189201787, 1.9853020622299218, 2.0167756278877973, 1.9694665700625396, 1.947542194975338, 1.9187722384319283, 1.9562135561083394, 2.0022435628516733, 2.013945035405369, 1.9943329258334448, 2.059597129746455, 2.0100162038484557]],
[[2.30846437097772, 2.4359642869050813, 2.3162267509627563, 2.526291288685951, 2.5910337075297636, 2.3842540339076432, 2.4693811544821633, 2.394231894572507, 2.3103815230438203, 2.2734535782151135, 2.5242104810666635, 2.27764076320977, 2.569086699904906, 2.4122442205935313, 2.3390552537252507, 2.256866573430023, 2.3541986262019376, 2.1859664962695784, 2.4077418648007023, 2.3980958968864035], [2.043984907527127, 2.0165215053815273, 1.9782215003256125, 2.1318233526576815, 2.2392563070534797, 2.047428871437107, 2.0744563961748264, 2.0511164966717472, 1.986561949506163, 1.9383226148597292, 2.1576664120875817, 2.003594195833769, 2.144323320134677, 1.9786430876381997, 2.0229900740722697, 2.0471815765370858, 2.2316849401093517, 1.9326178246623194, 2.0083591205177833, 2.0527424589646066]],
[[2.17079900590846, 2.4806107416983814, 2.653820440133195, 3.187031423657182, 2.875700546856189, 2.855657477461132, 3.5527171416007994, 3.0163775780489526, 2.8520286415314597, 2.4121956381078444, 3.098195170953898, 3.258356430546729, 2.2363277197505926, 2.770746074991716, 2.9290677949467305, 2.7786134942938667, 3.0307051539953123, 2.4755192616438055, 2.84556761172634, 2.221888158664993], [1.902720270941735, 1.83254804193308, 1.8212173312473383, 1.8899841807460351, 1.8567152131706242, 1.8702404677296534, 1.8578685935163095, 1.8830971905166516, 1.8973340980461304, 1.821885717345997, 1.897247079836042, 1.8625126491464459, 1.8687917554949458, 1.8990602800095313, 1.9066659443436467, 1.8840799785685673, 1.835837881111076, 1.9153809601930976, 1.831657951909525, 1.8737583073359494]],
[[2.0311805499937274, 2.9639356861848465, 2.8675155040900724, 3.4997705195320905, 2.780110954013694, 2.119632157777885, 2.9876098036942365, 3.8006543713336165, 3.011828332892164, 2.886818964333978, 2.710917207937638, 2.611161456908028, 2.445352394488135, 2.9062272947949266, 3.2749604212743444, 2.118537219566159, 3.382988573802876, 2.204843042788003, 3.4822695464539892, 2.4555196682835705], [1.690303900774121, 1.7963455720537966, 1.7420908169340563, 1.7446516974636532, 1.7087027033619957, 1.7752419625380929, 1.7164302632775814, 1.7527347774203623, 1.7097641264113885, 1.7169075035769226, 1.7611586347070922, 1.724119814123827, 1.714232243249767, 1.7856502531858502, 1.7335294903198688, 1.7239103201898593, 1.7684662339182737, 1.7593636649802946, 1.7118585924405048, 1.726227214490198]],
[[4.132778048168779, 3.3761641018043087, 3.5683744887350475, 1.9128468628567405, 3.8527851093861556, 2.2981947250254993, 3.853201455797011, 4.357481674768869, 3.8931586563074427, 3.4307660340819757, 1.909590735614929, 3.0973995464535786, 4.802834425861943, 4.577217932457659, 3.6895427038688724, 2.356523903170461, 4.251439236051537, 2.9407133644690964, 2.0848992069152166, 3.4392390822046366], [0.8027787852203456, 0.7575896781338559, 0.7770670126772649, 0.7691229516873522, 0.7548017251654942, 1.054395739701369, 1.0828866313569854, 1.0553820513542007, 1.0624225142545785, 1.1024396569383499, 1.123991189625993, 1.08681718045695, 1.0833891568651632, 1.15736746363531, 1.078247384903066, 1.0028503518615657, 1.0766571254137585, 1.053823963678581, 1.040818959860091, 1.085864146593737]],
[[3.903160819380223, 3.1269977439721135, 3.4162541447111465, 3.1411370966973657, 3.4324913896649267, 4.759896481618264, 2.243954577962971, 1.9107077106041674, 4.266543780735726, 1.1247309231276428, 1.1216210067199133, 2.4284018143044594, 4.167124140628104, 2.372150178003849, 3.2779830728100294, 4.611704475711701, 3.798555113378417, 4.090043510357762, 1.5279844487553234, 1.3841984920226906], [0.7565528187807422, 0.8112924020611376, 0.8042784218812256, 0.7750086756418613, 0.7626621947373982, 0.7668542803652268, 0.7870784182118236, 0.7655312694647107, 0.7559101323475892, 0.7874727500557921, 0.7941016658562875, 0.8095004134904888, 0.7924550921179017, 0.7503516665260654, 0.785082484155257, 0.8138708462910768, 0.7858609350663743, 0.7861738582932041, 0.7543601267394242, 0.7454228156393572]],
[[3.685003933005164, 2.9401097999364016, 3.23121431023925, 2.955075668688674, 3.3557894400285004, 3.2203863526317793, 3.0927596109330318, 3.709565038140232, 2.6656858063111115, 3.8642056296962846, 1.6517399481637327, 3.4528954361918895, 3.160410144143553, 4.607788956003375, 2.4329653174765875, 1.8225882652582452, 2.2106644316441284, 4.114694556566095, 2.601014514935369, 4.326839093906616], [1.0735381163021653, 1.0501741200983825, 1.0252217388584084, 1.0723175572216443, 1.1320768478782892, 1.0942891131446137, 1.0581452992051434, 1.075716053242795, 1.032771391432713, 1.0272901247041224, 1.1857025203289737, 1.1982183597446767, 1.2079332947475057, 1.2240040114302986, 1.2560075509684476, 1.1917805159060337, 1.1870391365046806, 1.2509422470028773, 1.3441062453760333, 1.2670730210041856]],
[[1.5669529952698376, 2.9905106989124444, 3.562317390170609, 3.7814308994533405, 3.3246296468116867, 1.9947418890648263, 3.7931603910553515, 3.805452287075046, 4.273321904427253, 1.9190271347828034, 2.108780901331392, 2.684551204869293, 4.099931238521929, 3.994910334947607, 1.785580831120945, 1.2050796670836637, 4.314847525897077, 3.8923812443602266, 3.771303219136158, 4.082226347515881], [1.2714933680545493, 1.1918191664712146, 1.2153753741593496, 1.2809886781996278, 1.2333718203981414, 1.1991915667252888, 1.2206771162690648, 1.235448826335843, 1.2088945843105265, 1.257836520811313, 1.2025743910802427, 1.2098183483106453, 1.168935826387985, 1.2205535234698077, 1.2155053766707822, 1.303027664441021, 1.331431669711204, 1.3642054149199405, 1.3449947870677172, 1.321613882416647]],
[[1.7476409059977713, 3.640943941202912, 3.389639485304402, 2.5682262879819766, 1.7037369299302256, 4.034157038311915, 2.363659987710484, 2.632715949032101, 3.6164186795825963, 2.476251667765769, 4.525025759753246, 4.03212716564465, 2.6582413142358603, 3.630234598564307, 1.9354912207451878, 2.9289324340515037, 3.8796646632632372, 4.0635720747500566, 3.968229536164992, 4.633380535762498], [1.370177349390382, 1.292559772961102, 1.2908721787010136, 1.3697314256845283, 1.3087400000362777, 1.3278594759689113, 1.344156937205196, 1.327896541416409, 1.3238475882359586, 1.3368238886983863, 1.341425867254603, 1.4209894782198713, 1.2961584301495526, 1.349039667663579, 1.2711300907802312, 1.3102947835520327, 1.3360916884812575, 1.346292718287076, 1.3113512635675804, 1.3308277049882045]],
[[3.703945281772142, 3.6946504397572015, 2.9819854872282017, 3.777318072764422, 4.251851957634419, 4.1049099820772845, 3.3161047868367493, 4.1211719665578475, 1.8653060340245573, 2.509107626002025, 4.561539680947411, 2.252431382383739, 3.929349599571145, 4.0575206459093, 4.229541898056227, 3.764940604621286, 2.6042239182229623, 4.106135025883253, 2.0246416588521567, 4.215162993292128], [1.4194651251873849, 1.3943503211462456, 1.4201955946911475, 1.3413953094186275, 1.3742581346971154, 1.3773742189626041, 1.359273160327744, 1.386849901034099, 1.4151823626637836, 1.3889423818024444, 1.425537519058165, 1.3771969864128295, 1.4455347441515214, 1.413817084327158, 1.3614731664745732, 1.3671150424402527, 1.442176964788955, 1.3952465908037255, 1.3439987904641486, 1.3892094456283246]],
[[3.4497490234859427, 2.8622105737037877, 3.6309212393640258, 4.086788445271122, 4.239862677200915, 3.5567568153137414, 3.9271848560918814, 3.651050046225412, 4.2566750753876805, 3.852319581920043, 3.733802564937096, 3.3576134671743096, 3.843996660543005, 2.887673358114272, 3.737241727130166, 3.1466410087624523, 3.680878610604215, 3.744260902069612, 1.8279249827912925, 3.017683286126016], [1.3418520941281806, 1.3250193560335901, 1.3349179788109986, 1.4615343386470339, 1.3890344572735562, 1.450437305545775, 1.4293857213351995, 1.505722970575844, 1.4420466988230212, 1.4298484726472958, 1.455844458869635, 1.4489020673180883, 1.380964602765016, 1.393060086550452, 1.425005202256631, 1.422658789804925, 1.409629904281084, 1.446555699882092, 1.3943491839137774, 1.463106144815362]],
[[4.825912761062593, 3.620128910867354, 3.5734940139382743, 3.8309695153136016, 2.3317848696702477, 3.9387420939593616, 2.585796091624812, 3.7530689011944705, 3.8649227537606548, 3.7791956328833027, 4.060220201196307, 2.5534277445879856, 1.6856990426373704, 2.085634820111752, 3.5703065939141885, 3.8714940192483907, 3.634711927884799, 2.029071849366746, 3.638125756369008, 4.105561873542407], [1.4465572392538757, 1.396184380801461, 1.3787785665430174, 1.4685285123907506, 1.430647184174049, 1.4177659763945663, 1.4411455962520134, 1.3953602117988508, 1.4171594675537942, 1.4666220624864654, 1.5207193171544087, 1.5453596954350208, 1.46848118590656, 1.4769744810361969, 1.4379022698930735, 1.491305390265963, 1.490197704295254, 1.4519878178779644, 1.457128563294945, 1.5066925594009841]],
[[2.680913609936137, 2.1881428237821767, 2.2053638059199248, 3.124508366192405, 3.394131331042537, 3.9040934021303104, 2.5980969194727033, 2.1139585952877327, 2.960073429234768, 3.786064810730645, 3.449760718850149, 2.074330570103662, 2.080174158072774, 3.7802294605072784, 3.6696628547466434, 4.176596925814581, 3.5037467910675177, 2.572108058413069, 4.024105581624729, 3.526587680711411], [1.4482488877983484, 1.4896658663882782, 1.5228682765294983, 1.4086531049609574, 1.4921213815242969, 1.4943809228103893, 1.5028794800021499, 1.4914693775250651, 1.4756000693181226, 1.5075281972068266, 1.493741112600543, 1.4324379311173827, 1.4897758116641011, 1.4284118422631606, 1.4739056017732528, 1.6133548077852147, 1.5511250109399035, 1.5280388249003434, 1.5154055216158109, 1.520057134306668]],
[[2.605672034918071, 2.470088246137587, 3.7064607609914173, 3.073385489127965, 3.313173104757367, 4.2187633562907045, 3.053863209425835, 2.018950349773135, 3.385831315073408, 3.0475879366771315, 3.5311387666637444, 3.2130791022571605, 1.9847260421795447, 4.270024013164229, 4.008582189279394, 2.2460505394349854, 2.5902470150617383, 3.3120280178543022, 4.0247678855842555, 2.0736519896976047], [1.5519870032034364, 1.4840612608305062, 1.617618501229649, 1.5330200457616125, 1.5351448552411222, 1.5063950879352885, 1.5311630001020355, 1.4953389267675505, 1.5193282238431782, 1.5497475431546401, 1.5142578605197576, 1.4828379184994325, 1.5566839659321605, 1.517524132424232, 1.6059038459356088, 1.4781425872875629, 1.55306050806555, 1.538443750802371, 1.5202220892193632, 1.566141642462447]],
[[2.1251891243387093, 3.674233692776258, 3.956095820200619, 2.6000516085424015, 3.6858601788866334, 3.713394129232431, 3.7168934443461286, 3.599187366908841, 4.009216900636766, 3.5895308892021927, 3.6290486449687176, 4.413802905510104, 1.8645180517040405, 3.81207030879869, 2.1818729189838866, 3.924232152411025, 3.8920301672822784, 3.068379778236179, 1.8397979464512357, 3.3747829955181965], [1.5773921495813814, 1.5488340509276841, 1.5895011741384208, 1.604689683342734, 1.564208905220619, 1.5826040463315598, 1.575850812295573, 1.586229154981369, 1.597502170120181, 1.5822961043196326, 1.5464129329150598, 1.6064376992766018, 1.5730265924429012, 1.5218955585950635, 1.605020192193145, 1.6635755828050898, 1.5749429331182905, 1.6006639498574229, 1.588361020487592, 1.569289448967022]],
[[2.9375214269925727, 3.4138431162387595, 4.408866722745872, 3.589409028583049, 4.130734809434325, 3.4105856783351585, 3.91673683844475, 4.304039649921251, 2.2027349478235454, 4.144556860719871, 3.2480768846541785, 3.7156629111993213, 3.4279117891528372, 3.2137674724765697, 3.2984787520730245, 3.8212548969672353, 4.195177980245274, 3.252536894900187, 3.77510065403077, 2.7147628418134224], [1.605804968807904, 1.608400390510395, 1.5994119144220897, 1.5862151686545582, 1.5710753999418456, 1.6478490599012707, 1.5800662479615815, 1.631089030829981, 1.6060943706203399, 1.6209569528202097, 1.6247815920370383, 1.6420773155234758, 1.6297501429299646, 1.658291518282065, 1.6388012302522832, 1.6046837989359228, 1.651209307442816, 1.610477329476418, 1.6363578067602176, 1.5860871656682778]],
[[4.062959074355161, 2.4997644886490846, 1.8576372490042392, 2.8324617790329114, 4.192788535239488, 3.0488420099850586, 3.2070816546472303, 3.2515685277829536, 3.261597997188946, 3.4391876957269445, 3.207212788216265, 3.766356986131174, 4.004138888385777, 3.7359815854860305, 2.1817459544311717, 3.6877574946219602, 2.209276777969228, 2.169019719767558, 3.6490303746383987, 3.4453651838217954], [1.615462004830436, 1.5801433791597557, 1.5650650005338347, 1.6471463980931664, 1.6145893067328576, 1.6564850648719107, 1.653065133334065, 1.6094875975654646, 1.6419910686558759, 1.6297719254172558, 1.6657031725523104, 1.6265517891502697, 1.6324170448014779, 1.6438685827794826, 1.6506708777758057, 1.655800370864628, 1.670206045979184, 1.6864887915557993, 1.6630635679353536, 1.672513974481568]],
[[3.3208178610114607, 3.556283506049092, 2.6336573438578172, 3.3473220746842904, 3.902269730125125, 1.9056505792402896, 3.635557483195467, 3.4688389338880454, 3.752182108589029, 4.026100357924521, 2.002345802823391, 2.194849247837751, 3.427613931671977, 3.703929685363112, 3.1182504865943677, 2.9557253978309848, 3.7636153333023548, 3.6408984544739025, 3.8035115707646345, 2.443773651929538], [1.6906528427659826, 1.6957232766506731, 1.6498718552719132, 1.6447635215946246, 1.6738692745851804, 1.6396739820161168, 1.6738723255232268, 1.678811208057696, 1.655560980059005, 1.7038796620756576, 1.7222132874742901, 1.6389878836635168, 1.6952677366960862, 1.649003885952329, 1.6336592587269514, 1.6824776386539133, 1.6915031232019067, 1.7104947761803855, 1.7195815631071676, 1.6645882906610745]],
[[3.8191453514281637, 3.9489794891534427, 3.3544927152179973, 3.179357305479818, 3.782123226187971, 3.850077249255284, 2.0630204218712467, 1.9533613168732877, 2.7516678300696626, 3.865936429204303, 2.920824661888608, 3.9785131798043256, 3.729223059762138, 2.338568920041232, 3.469083406758888, 3.9997073970197974, 2.277639807660014, 3.6791964523189633, 3.5902609248778896, 3.4899059232423335], [1.7094896356537685, 1.7369275132964317, 1.6882896694137741, 1.722215100189337, 1.691721153322507, 1.7067002686314021, 1.7393968274559, 1.6746427734452434, 1.6868551191680643, 1.69433905895392, 1.7105095353564779, 1.6949039882776211, 1.7001404385427563, 1.6677132246962292, 1.7158020287201783, 1.7102341557328902, 1.7554622990268718, 1.6852014004112483, 1.700093613156155, 1.6858447760179602]]]

In [ ]:
optimal_boundary(no_modified, modified, 0.2, 0.005)

In [ ]:
!pip install --upgrade plotly

In [ ]:
supermega_not_mod = []
supermega_mod = []
for a, b in supermega:
    supermega_not_mod += b
    supermega_mod += a

In [ ]:
'''

CODE FOR HISTOGRAMS

'''


import plotly.express as px
import pandas as pd
import numpy as np

no_modified = # INSERT HERE RESULTS OF NON MODIFIED
modified = # INSERT HERE RESULTS OF MODIFIED

boundary, f1, tp, fp = optimal_boundary(no_modified, modified, 0.125, 0.005)

print('Boundary:', boundary)
print('F1:', f1)
print('tp:', tp)
print('fp:', fp)

df = pd.DataFrame(dict(
    Type=np.concatenate((["Original test"]*len(no_modified), ["Modified test"]*len(modified))), 
    Data  =np.concatenate((no_modified,modified))
))


fig = px.histogram(df, x="Data", color="Type", barmode="overlay", nbins=200).for_each_trace(lambda t: t.update(name=t.name.replace("Type=","")))

fig.update_layout(width=600, height=500)

fig.update_layout(
    legend=dict(
        x=0.65,
        y=0.9,
        traceorder="normal",
    )
)

fig.update_layout(
    xaxis_title=r'$\text{d}_{C}$',
    yaxis_title=r'',
)

fig.update_yaxes(range=[0, 850])
fig.update_xaxes(range=[0,0.4])

fig.update_layout(showlegend=False)

fig.add_vline(x = boundary)

fig.show()